Preprocess

In [5]:
import os
import torchvision.transforms as transforms
from PIL import Image
from torch.utils.data import DataLoader, Dataset
import torch #touhid

device = torch.device("cuda")
class SignatureDataset(Dataset):
    def __init__(self, data_folder, transform=None):
        self.data_folder = data_folder
        self.image_list = os.listdir(data_folder)
        self.transform = transform

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx):
        image_name = os.path.join(self.data_folder, self.image_list[idx])

        if image_name.endswith(".png"):
            image = Image.open(image_name)

            # Convert the grayscale image to 1-channel grayscale
            image = image.convert("L")

            if self.transform:
                image = self.transform(image)

            return image


def preprocess_data(data_folder, image_size=(128, 128), batch_size=32, test=False, device=None):
    data_transform = transforms.Compose([
        transforms.Resize(image_size),  # resizes images
        transforms.ToTensor(),  # converts to tensor
        # transforms.Normalize(mean=[0.5], std=[0.5]),  # normalization
    ])

    signature_dataset = SignatureDataset(data_folder, transform=data_transform)
    # Filter out None elements from the signature_dataset
    signature_dataset = [signatures.to(device) for signatures in signature_dataset if signatures is not None]

    if test:
        # batch_size = len(signature_dataset)
        signature_dataloader = signature_dataset
    else:
        signature_dataloader = DataLoader(signature_dataset, batch_size=batch_size, shuffle=False)

    return signature_dataloader


Triplet Loss

In [6]:
import torch
import torch.nn as nn


class TripletLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(TripletLoss, self).__init__()
        self.margin = margin

    def forward(self, anchor_output, positive_output, negative_output):
        distance_positive = torch.norm(anchor_output - positive_output, p=2, dim=1)
        distance_negative = torch.norm(anchor_output - negative_output, p=2, dim=1)
        losses = torch.relu(distance_positive - distance_negative + self.margin)
        return torch.mean(losses)


Siamese Net

In [7]:
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer


class SiameseNetwork(nn.Module):
    def __init__(self, embedding_dim, batch_size, num_heads, num_layers, dropout=0.1):
        super(SiameseNetwork, self).__init__()
        self.embedding_dim = embedding_dim
        self.batch_size = batch_size

        # Define the CNN architecture
        self.cnn = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
        )

        # Define the architecture of the Siamese network using Transformer
        self.transformer_encoder = TransformerEncoder(
            TransformerEncoderLayer(d_model=embedding_dim, nhead=num_heads, dropout=dropout),
            num_layers=num_layers
        )

        # Calculate the number of features after Transformer layers
        self.transformer_features = embedding_dim  # This can be adjusted based on your requirements

        # Fully connected layer for producing the final embeddings
        self.fc1 = nn.Linear(self.transformer_features, embedding_dim)

    def forward_once(self, x):
        # x = x.view(self.batch_size, 64, 256)
        # x = x.transpose(0, 1)  # Transpose dimensions to match Transformer input format
        # x = self.transformer_encoder(x)
        # x = x.transpose(0, 1)  # Transpose dimensions back to batch-first
        # x = x.mean(dim=1)  # Average over sequence length to get a fixed-size representation
        # x = F.relu(self.fc1(x))

        x = self.cnn(x)
        # print(x.shape)

        x = x.flatten(2).transpose(1, 2)  # Flatten the image features and transpose for Transformer
        # print(x.shape)
        # x = x.permute(0, 2, 1)
        # print(x.shape)
        x = self.transformer_encoder(x.to(x.device))
        # x = self.transformer_encoder(x)
        x = x.mean(dim=1)  # Average over sequence length to get a fixed-size representation
        x = F.relu(self.fc1(x))
        return x

    def forward(self, anchor, positive, negative):
        anchor_output = self.forward_once(anchor)
        positive_output = self.forward_once(positive)
        negative_output = self.forward_once(negative)
        return anchor_output, positive_output, negative_output

    def count_parameters(self):
        return sum(p.numel() for p in self.parameters() if p.requires_grad)

    def __str__(self):
        return f"SiameseNetwork(embedding_dim={self.embedding_dim}, num_params={self.count_parameters()})"


Train and Test

In [8]:
import torch
import random
# from triplet_loss import TripletLoss
import time
import torch.nn.functional as f
from sklearn.metrics import precision_recall_fscore_support, accuracy_score


def train_siamese_network(siamese_net, anchor_dataloader, positive_dataloader, negative_dataloader, num_epochs=10):
    # Initialize the Siamese network and TripletLoss
    criterion = TripletLoss(margin=0.5)
    optimizer = torch.optim.Adam(siamese_net.parameters(), lr=0.001)

    # Training loop
    start_time = time.time()
    for epoch in range(num_epochs):
        running_loss = 0.0

        siamese_net.train()  # Set the model to training mode
        for batch_idx, (anchors, positives, negatives) in enumerate(
                zip(anchor_dataloader, positive_dataloader, negative_dataloader)):
            optimizer.zero_grad()

            # print(anchors.shape)
            # Reshape the tensors to [batch_size, num_channels * height * width]
            # anchors = anchors.view(anchors.size(0), -1)  # The -1 automatically calculates the necessary size
            # positives = positives.view(positives.size(0), -1)
            # negatives = negatives.view(negatives.size(0), -1)

            # Assuming anchors, positives, and negatives are your input tensors
            # print("Anchors shape:", anchors.size())
            # print("Positives shape:", positives.size())
            # print("Negatives shape:", negatives.size())

            anchors = anchors.to(device)
            positives = positives.to(device)
            negatives = negatives.to(device)

            # Forward pass through the Siamese Network
            anchor_output, positive_output, negative_output = siamese_net(anchors, positives, negatives)

            # Compute the triplet loss
            loss = criterion(anchor_output, positive_output, negative_output)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            # print(loss.item())

        # print(len(anchor_dataloader))
        epoch_loss = running_loss / len(anchor_dataloader)  # Calculate the average loss for the epoch
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}")

    end_time = time.time()
    total_time = end_time - start_time
    print(f"Total training time: {total_time:.2f} seconds")


def test_siamese_network(siamese_net, test_dataloader):
    siamese_net.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        correctly_predicted = 0
        incorrectly_predicted = 0
        all_true_labels = []
        all_predicted_labels = []

        for anchors, positives, negatives in test_dataloader:

            anchors = anchors.unsqueeze(0)  # Add batch dimension
            positives = positives.unsqueeze(0)  # Add batch dimension
            negatives = negatives.unsqueeze(0)  # Add batch dimension
            # print(anchors.shape)

            anchors = anchors.to(device)
            positives = positives.to(device)
            negatives = negatives.to(device)
            # anchor_signature = anchor_signature.view(1, -1)  # Reshape the anchor signature
            # test_signature = test_signature.view(1, -1)  # Reshape the test signature

            # Generate random number for positive and negative
            random_number = random.randint(0, 1)

            if random_number == 0:
                anchor_output = siamese_net.forward_once(anchors)
                test_output = siamese_net.forward_once(negatives)
            else:
                anchor_output = siamese_net.forward_once(anchors)
                test_output = siamese_net.forward_once(positives)

            # Calculate the distance between anchor and test signatures
            distance = f.pairwise_distance(anchor_output, test_output)

            # If the distance is below a certain threshold, consider the test signature as genuine (positive)
            # Otherwise, consider it as forged (negative)
            threshold = 0.5
            predicted_label = 1 if distance < threshold else 0

            all_true_labels.append(random_number)
            all_predicted_labels.append(predicted_label)

            # print("Distance: {:.4f}".format(distance.item()))
            # print(f"Predicted Label: {predicted_label}")
            # print(f"Actual Label: {random_number}")

            if predicted_label == random_number:
                correctly_predicted += 1
            else:
                incorrectly_predicted += 1


        print(f"Total Correctly Predicted: {correctly_predicted}")
        print(f"Total Incorrectly Predicted: {incorrectly_predicted}")

        # Calculate precision, recall, f1-score, and accuracy
        precision, recall, f1, _ = precision_recall_fscore_support(all_true_labels, all_predicted_labels,
                                                                   average='binary')
        accuracy = accuracy_score(all_true_labels, all_predicted_labels)

        print(f"Precision: {precision:.2f}")
        print(f"Recall: {recall:.2f}")
        print(f"F1-score: {f1:.2f}")
        print(f"Accuracy: {accuracy:.2%}")




In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Main

In [10]:
def main():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f'Selected Device: {device}')
    train_anchor_folder_path = "/content/drive/My Drive/datasets_713/triplet_dataset/train/anchor"
    train_positive_folder_path = "/content/drive/My Drive/datasets_713/triplet_dataset/train/positive"
    train_negative_folder_path = "/content/drive/My Drive/datasets_713/triplet_dataset/train/negative"

    # validate_anchor_folder_path = "/content/drive/My Drive/hsv_data/triplet_dataset/validate/anchor"
    # validate_positive_folder_path = "/content/drive/My Drive/hsv_data/triplet_dataset/validate/positive"
    # validate_negative_folder_path = "/content/drive/My Drive/hsv_data/triplet_dataset/validate/negative"

    test_anchor_folder_path = "/content/drive/My Drive/datasets_713/triplet_dataset/test/anchor"
    test_positive_folder_path = "/content/drive/My Drive/datasets_713/triplet_dataset/test/positive"
    test_negative_folder_path = "/content/drive/My Drive/datasets_713/triplet_dataset/test/negative"

    image_size = (128, 128)
    batch_size = 16
    num_epochs = 10
    desired_embedding_dim = 128
    num_heads = 4
    num_layers = 4

    # preprocess data
    train_anchor_dataloader = preprocess_data(train_anchor_folder_path, image_size=image_size, batch_size=batch_size)
    train_positive_dataloader = preprocess_data(train_positive_folder_path, image_size=image_size, batch_size=batch_size)
    train_negative_dataloader = preprocess_data(train_negative_folder_path, image_size=image_size, batch_size=batch_size)

    # validate_anchor_dataloader = preprocess_data(validate_anchor_folder_path, image_size=image_size, batch_size=batch_size)
    # validate_positive_dataloader = preprocess_data(validate_positive_folder_path, image_size=image_size, batch_size=batch_size)
    # validate_negative_dataloader = preprocess_data(validate_negative_folder_path, image_size=image_size, batch_size=batch_size)
    #
    test_anchor_dataloader = preprocess_data(test_anchor_folder_path, image_size=image_size, test=True)
    test_positive_dataloader = preprocess_data(test_positive_folder_path, image_size=image_size, test=True)
    test_negative_dataloader = preprocess_data(test_negative_folder_path, image_size=image_size, test=True)
    test_dataloader = zip(test_anchor_dataloader, test_positive_dataloader, test_negative_dataloader)
    # print(test_anchor_dataloader[0])

    num_batches = len(train_anchor_dataloader)
    print(f"Number of batches in signature_dataloader: {num_batches}")

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f'Selected Device: {device}')
    siamese_net = SiameseNetwork(embedding_dim=desired_embedding_dim, batch_size=batch_size, num_heads=num_heads,
                                 num_layers=num_layers).to(device)   # Use the SiameseNetwork with Transformer

    # Train the Siamese network
    train_siamese_network(siamese_net, train_anchor_dataloader, train_positive_dataloader, train_negative_dataloader,
                            num_epochs)

    # Validate the Siamese network

    # Test the Siamese network
    test_siamese_network(siamese_net, test_dataloader)

    # SHAP
    # sx.explain_shap_2(siamese_net, train_anchor_dataloader, train_positive_dataloader, train_negative_dataloader)  # Assuming siamese_net is your trained model


if __name__ == "__main__":
    main()


Selected Device: cuda
Number of batches in signature_dataloader: 75
Selected Device: cuda
Epoch 1/10, Loss: 0.0487
Epoch 2/10, Loss: 0.0000
Epoch 3/10, Loss: 0.0000
Epoch 4/10, Loss: 0.0000
Epoch 5/10, Loss: 0.0000
Epoch 6/10, Loss: 0.0000
Epoch 7/10, Loss: 0.0000
Epoch 8/10, Loss: 0.0000
Epoch 9/10, Loss: 0.0000
Epoch 10/10, Loss: 0.0000
Total training time: 260.59 seconds
Total Correctly Predicted: 78
Total Incorrectly Predicted: 22
Precision: 1.00
Recall: 0.53
F1-score: 0.69
Accuracy: 78.00%
